In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.cluster
import math as m
import sklearn.neighbors
import sklearn.mixture
import sklearn.ensemble
import glob
import pickle
import os

# Making a data set using the ZTF data
# I am adding some conversions here for events which are either "candidates" or have <5 objects
conversions = {'SN Ia-91T-like':'SN Ia-91T', 'SN IIn-pec':'SN IIn', 'TDE-H-He':'TDE', 'SN IIP':'SN II', 
               'SN IIL': 'SN II', 'SN Iax[02cx-like]': 'SN Iax'}
def make_dataset(load=False):
    # to do - eventually, I do want to save/load a dataset...
    if not load:
        metadata, all_labels = np.loadtxt('/Users/pmark/OneDrive/Desktop/Research Materials/Training set/training_set_dynesty.csv',usecols=(0,1), 
                                          delimiter=',', skiprows=1, dtype=str, unpack=True)
        
        gind = np.where((all_labels !='Galaxy') & (all_labels !='Other') &  (all_labels !='M dwarf') & (all_labels !='SN I'))
        metadata = metadata[gind]
        all_labels = all_labels[gind]
        all_dat = np.zeros((len(all_labels),15))
        names = np.array([])
        for key in conversions:
            gind = np.where(all_labels == key)
            all_labels[gind] = conversions[key]
        print(np.unique(all_labels, return_counts=True))

        for i, md in enumerate(metadata):
            one_sn_data = np.load('/Users/pmark/OneDrive/Desktop/Research Materials/Training set/dynesty_fits/'+md+'_eqwt_dynesty.npz')
            all_dat[i] = np.mean(one_sn_data['arr_0'],axis=0)
            names = np.append(str(md), names)
        return(all_labels,all_dat, names)

labels,all_features, names = make_dataset()
print(names)
all_features = np.log10(all_features[:, [2,4,5,6]])
all_features = np.delete(all_features, [0,3], 1)

# Loading in pre-existing GMM and IF files

# Here I load in the existing mixture model and fit it to the new training set
means = np.load("/Users/pmark/OneDrive/Desktop/Research Materials/Training set/Phillips_deSoto_gm_fit_means.npy")
weights = np.load('/Users/pmark/OneDrive/Desktop/Research Materials/Training set/Phillips_deSoto_gm_fit_weights.npy')
covar = np.load('/Users/pmark/OneDrive/Desktop/Research Materials/Training set/Phillips_deSoto_gm_fit_covariances.npy')        
loaded_gm = sklearn.mixture.GaussianMixture(n_components = len(means), random_state = 0, covariance_type = 'full')
loaded_gm.means_ = means
loaded_gm.covariances_ = covar
loaded_gm.weights_ = weights
loaded_gm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
loaded_gmm = loaded_gm.fit(all_features)

# Here I load the Isolation forest model and fit it to the new training set.
IF_file = np.load('/Users/pmark/OneDrive/Desktop/Research Materials/Training set/Phillips_IsolationForest.npy', allow_pickle = True)
loaded_IF = sklearn.ensemble.IsolationForest(n_estimators = len(IF_file))
loaded_if = loaded_IF.fit(all_features)

# Here I calculate the anomaly scores for the mixture model and isolation forest

anom_scores_GMM = -1 * loaded_gmm.score_samples(all_features)
anom_scores_IF = -1 * loaded_IF.score_samples(all_features)

# GMM and IF anomaly scores by SN classification
snia_anom_scoresgmm =np.array([]) 
snibc_anom_scoresgmm = np.array([]) 
snii_anom_scoresgmm = np.array([]) 
snii_bn_anom_scoresgmm =np.array([]) 
slsn_anom_scoresgmm = np.array([]) 
other_anom_scoresgmm = np.array([])

snia_anom_scoresif =np.array([]) 
snibc_anom_scoresif = np.array([]) 
snii_anom_scoresif = np.array([]) 
snii_bn_anom_scoresif =np.array([]) 
slsn_anom_scoresif = np.array([]) 
other_anom_scoresif = np.array([])

# Here I iteratively run through 
for i in range(len(labels)):
    # Sorting all of the SNe that are type Ia
    if str(labels[i]) == "SN Ia" or str(labels[i]) == "SN Ia-91T" or str(labels[i]) == "SN Ia-91bg-like"  or str(labels[i]) == "SN Ia-CSM" or str(labels[i]) == "SN Ia-SC" or str(labels[i]) == "SN Ia-pec" or str(labels[i]) == "SN Iax":
        snia_anom_scoresgmm = np.append(anom_scores_GMM[i], snia_anom_scoresgmm)
        snia_anom_scoresif = np.append(anom_scores_IF[i], snia_anom_scoresif)
    # Sorting all of the type Ib/c SNe
    elif str(labels[i]) == "SN Ib" or str(labels[i]) == "SN Ic" or str(labels[i]) == "SN Ib/c" or str(labels[i]) == "SN Ibn" or str(labels[i]) == "SN Icn" or str(labels[i]) == "SN Ib-pec" or str(labels[i]) == "SN Ic-BL" or str(labels[i]) == "SN Ic-Ca-rich" or str(labels[i]) == "SN Ic-pec":
        snibc_anom_scoresgmm = np.append(anom_scores_GMM[i], snibc_anom_scoresgmm)
        snibc_anom_scoresif = np.append(anom_scores_IF[i], snibc_anom_scoresif)
    # Sorting all of the type II's that are not IIb or IIn's
    elif str(labels[i]) == 'SN II' or str(labels[i]) == 'SN II-pec':
        snii_anom_scoresgmm = np.append(anom_scores_GMM[i], snii_anom_scoresgmm)
        snii_anom_scoresif = np.append(anom_scores_IF[i], snii_anom_scoresif)
    # Sorting Type IIb's and IIn's
    elif str(labels[i]) == 'SN IIb' or str(labels[i]) == 'SN IIn':
        snii_bn_anom_scoresgmm = np.append(anom_scores_GMM[i], snii_bn_anom_scoresgmm)
        snii_bn_anom_scoresif = np.append(anom_scores_IF[i], snii_bn_anom_scoresif)
    # Sorting SLSN's
    elif str(labels[i]) == 'SLSN-I' or str(labels[i]) == 'SLSN-II':
        slsn_anom_scoresgmm = np.append(anom_scores_GMM[i], slsn_anom_scoresgmm)
        slsn_anom_scoresif = np.append(anom_scores_IF[i], slsn_anom_scoresif)
    # Sorting all other light curves
    elif str(labels[i]) == 'AGN' or  str(labels[i]) == 'CV' or  str(labels[i]) == 'ILRT' or  str(labels[i]) == 'LBV' or  str(labels[i]) == 'LRN' or  str(labels[i]) == 'Nova' or  str(labels[i]) == 'QSO' or  str(labels[i]) == 'SN' or  str(labels[i]) == 'TDE' or  str(labels[i]) == 'Varstar':
        other_anom_scoresgmm = np.append(anom_scores_GMM[i], other_anom_scoresgmm)
        other_anom_scoresif = np.append(anom_scores_IF[i], other_anom_scoresif)
        
# Calculating the mean anomaly scores per population
snia_meanscore_GMM = np.mean(snia_anom_scoresgmm)
snia_meanscore_IF = np.mean(snia_anom_scoresif)

snibc_meanscore_GMM = np.mean(snibc_anom_scoresgmm)
snibc_meanscore_if = np.mean(snibc_anom_scoresif)

snii_meanscore_GMM = np.mean(snii_anom_scoresgmm)
snii_meanscore_IF = np.mean(snii_anom_scoresif)

sniibn_meanscore_GMM = np.mean(snii_bn_anom_scoresgmm)
sniibn_meanscore_IF = np.mean(snii_bn_anom_scoresif)

slsn_meanscore_GMM = np.mean(slsn_anom_scoresgmm)
slsn_meanscore_IF = np.mean(slsn_anom_scoresif)

other_meanscore_GMM = np.mean(other_anom_scoresgmm)
other_meanscore_IF = np.mean(other_anom_scoresif)

print(snia_meanscore_GMM, snia_meanscore_IF, snibc_meanscore_GMM, snibc_meanscore_if, snii_meanscore_GMM, snii_meanscore_IF, sniibn_meanscore_GMM, sniibn_meanscore_IF, slsn_meanscore_GMM, slsn_meanscore_IF, other_meanscore_GMM, other_meanscore_IF)

# Most Anomalous item in the GMM
print(np.max(anom_scores_GMM), anom_scores_IF[np.where(np.max(anom_scores_GMM))[0]] , labels[np.where(np.max(anom_scores_GMM))[0]])

# Most anomalous item in the IF
print(anom_scores_GMM[np.where(np.max(anom_scores_IF))[0]], np.max(anom_scores_IF) , labels[np.where(np.max(anom_scores_IF))[0]])


# Getting names of top anomaly for GMM using the features of the object
anom_features_gmm = all_features[np.where(np.max(anom_scores_GMM))[0]]
name_gmm_anom = names[np.where(np.max(anom_scores_GMM))[0]]
print(name_gmm_anom)

# Getting names of top anomaly for IF using the features of the object
anom_features_if = all_features[np.where(np.max(anom_scores_IF))[0]]
name_if_anom = names[np.where(np.max(anom_scores_IF))[0]]
print(name_if_anom)


# Finding the threshold scores for the

(array(['AGN', 'CV', 'ILRT', 'LBV', 'LRN', 'Nova', 'QSO', 'SLSN-I',
       'SLSN-II', 'SN', 'SN II', 'SN II-pec', 'SN IIb', 'SN IIn', 'SN Ia',
       'SN Ia-91T', 'SN Ia-91bg-like', 'SN Ia-CSM', 'SN Ia-SC',
       'SN Ia-pec', 'SN Iax', 'SN Ib', 'SN Ib-pec', 'SN Ib/c', 'SN Ibn',
       'SN Ic', 'SN Ic-BL', 'SN Ic-Ca-rich', 'SN Ic-pec', 'SN Icn', 'TDE',
       'Varstar'], dtype='<U17'), array([  47,  103,    1,    7,    1,   14,    4,   89,   49,   11,  957,
          4,   58,  196, 4173,  157,   24,   15,    3,   32,   12,   82,
          3,   14,   18,   92,   49,    1,    1,    2,   51,    8],
      dtype=int64))
['ZTF23aaetmwj' 'ZTF23aacjeou' 'ZTF23aacdnjz' ... 'ZTF23aaicagr'
 'ZTF23aaikcdx' 'ZTF23aaitimz']


c:\Users\pmark\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\pmark\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator IsolationForest from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


-1.9141448026491967 0.38613173510599885 -0.1323904130609125 0.447085940768306 1.3942834111714482 0.5258099231107676 1.2173300307554353 0.515399398137619 1.7293400913023902 0.5381560044030423 1.275369563097877 0.5198052312061633
7.733317836371409 [0.39139649] ['SN Ia']
[-2.16972373] 0.7895900863757621 ['SN Ia']
['ZTF23aaetmwj']
['ZTF23aaetmwj']
